In [1]:
IMAGE_DIMS = (50,34,50,1)
OUTPUT_DIR = '../../output/train-local/'
INPUT_DIR = '../../input/stage1_prepared_some_even/'
DATASET_NAME_SUFFIX = '-centered-rotated'
# LOAD_MODEL_FILE = None
LOAD_MODEL_FILE = OUTPUT_DIR + 'tf-checkpoint-best7826'

In [2]:
import csv
import h5py
import numpy as np # linear algebra
import os
import logging

import tflearn
from tflearn.layers.core import *
from tflearn.layers.conv import *
from tflearn.data_utils import *
from tflearn.layers.normalization import *
from tflearn.layers.estimator import regression

from modules.logging import logger
import modules.logging
import modules.lungprepare as lungprepare
import modules.utils as utils
import modules.cnn as cnn
from modules.utils import Timer

In [3]:
def start_training(model, input_dir, dataset_name_suffix, image_dims, output_dir):

    utils.mkdirs(output_dir, recreate=False)

    modules.logging.setup_file_logger(output_dir + 'out.log')
    
    dataset_path = utils.dataset_path(input_dir, 'test' + DATASET_NAME_SUFFIX, IMAGE_DIMS)
    with h5py.File(dataset_path, 'r') as train_hdf5:
        X = train_hdf5['X']
        Y = train_hdf5['Y']
        logger.info('X shape ' + str(X.shape))
        logger.info('Y shape ' + str(Y.shape))

        dataset_path = utils.dataset_path(input_dir, 'validate' + dataset_name_suffix, image_dims)
        with h5py.File(dataset_path, 'r') as validate_hdf5:
            X_validate = validate_hdf5['X']
            Y_validate = validate_hdf5['Y']
            logger.info('X_validate shape ' + str(X_validate.shape))
            logger.info('Y_validate shape ' + str(Y_validate.shape))

            logger.info('Starting CNN training...')
            model.fit(X, Y, validation_set=(X_validate, Y_validate), 
                      shuffle=True, batch_size=10, n_epoch=2,
                      show_metric=True, 
                      run_id='simplest1-'+str(image_dims))

In [ ]:
logger.info('Prepare CNN for training')
network = cnn.net_simplest1(IMAGE_DIMS)
model = cnn.prepare_cnn_model(network, OUTPUT_DIR, model_file=None)

2017-02-22 03:26:52,824 INFO Prepare CNN for training
2017-02-22 03:26:52,941 INFO Prepare CNN
2017-02-22 03:26:52,942 INFO Preparing output dir
2017-02-22 03:26:52,943 INFO Initializing network...


In [ ]:
logger.info('Train CNN')
start_training(model, INPUT_DIR, DATASET_NAME_SUFFIX, IMAGE_DIMS, OUTPUT_DIR)
model.save(OUTPUT_DIR + 'final')
logger.info('==== ALL DONE ====')

2017-02-22 03:26:54,419 INFO Train CNN
2017-02-22 03:26:54,422 INFO X shape (111, 50, 34, 50, 1)
2017-02-22 03:26:54,423 INFO Y shape (111, 2)
2017-02-22 03:26:54,424 INFO X_validate shape (10, 50, 34, 50, 1)
2017-02-22 03:26:54,425 INFO Y_validate shape (10, 2)
2017-02-22 03:26:54,426 INFO Starting CNN training...


---------------------------------
Run id: simplest1-(50, 34, 50, 1)
Log directory: ../../output/train-local/tf-logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.


2017-02-22 03:26:55,872 INFO Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.


---------------------------------
Training samples: 111
Validation samples: 10
--


In [ ]:
logger.info('Evaluate model from dataset')
dataset_path = utils.dataset_path(INPUT_DIR, 'test', IMAGE_DIMS)
evaluate_dataset(dataset_path, model):